In [7]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec


In [20]:

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "rag"

if index_name not in pc.list_indexes().names():
    pc.create_index(
    name= index_name, dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [22]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Computing',
  'stars': 4,
  'review': 'Dr. Smith is very knowledgeable and makes the lectures interesting. However, the exams are tough and require a lot of preparation.'},
 {'professor': 'Dr. Emily Johnson',
  'subject': 'Digital Logic Design',
  'stars': 5,
  'review': 'Dr. Johnson is an amazing professor! She explains concepts very clearly and provides plenty of resources for practice. Highly recommended.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Data Structures and Algorithms',
  'stars': 3,
  'review': "The course content is good, but Dr. Brown's teaching style can be a bit dry. Sometimes it’s hard to stay engaged during his lectures."},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Analog Integrated Circuits',
  'stars': 2,
  'review': "Dr. Lee is very strict and her exams are really difficult. It's hard to get a good grade in her class unless you spend a lot of time studying on your own."},
 {'professor': 

In [55]:

processed_data=[]
client = genai

for review in data['reviews']:
    result = genai.embed_content(
    model="models/text-embedding-004",
    content= review['review'],
)

processed_data.append({
    "values": list(result),
    "id": review["professor"],
    "metadata": {
        "review": review["review"],
        "subject": review["subject"],
        "stars": review["stars"]
    } 
})


In [60]:

index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace= "ns1"
)

TypeError: Column `values` is expected to be a list of floats